<img src="https://i.imgur.com/6U6q5jQ.png"/>
_____

<a id='home'></a>

# Merging and Choroplet maps

The merging process seems like a simple one: you take two tables and make one.
However, in this session I am going to show you a case where several issues arise on your way to merge tables, where one of them will be a map.

I will cover some important processes for tables:

* [Appending](#appending)
* [Reshaping](#reshaping)
* [Scaling](#scaling)

And finally:
* [Merging Data Frames](#merging)
* [Merging GEO-Data Frames and Data Frames](#geomerging)


<a id='appending'></a>

## Appending

As the name implies, this process binds DFs into one, that is, one or more DFs will be put below or on top of another DF. Appending can be done when you fulfill these requisites:
1. All the DFs  share the same column names.
2. All the DFs  columns are in the same location.

Note that it is better if the columns share the same data types. But if they don't, you can solve that during the formatting process.

Let's visit this website: https://fundforpeace.org/what-we-do/country-risk-and-fragility-data/

There, you will find several excel files with the _Fragile States Index_ per year. Please, create folder **worldData**, and inside of it a folder **fragility**, where you will download the excel files from 2013 to 2021. My folder looks like this:

![](imageForNotebooks/fragilityFiles.jpg)

How can we see all the files in a folder?

In [ ]:
# Import libraries
import os
import glob
import pandas as pd

path = os.path.join('worldData','fragility','*.xlsx') # xlsx files in the folder
excel_files_names = glob.glob(path) #file names using pyhton's glob

# see the file names
excel_files_names

We will save each data frame in a list: **allDFs**. We will use pandas, but we need **openpyxl** and **xlrd** (for Excel) before doing this operation:

In [ ]:
allDFs=[] # all XLSX will be here!

import pandas as pd

for fileName in excel_files_names:
    currentFile=pd.read_excel(fileName)
    allDFs.append(currentFile)

Does every data frame in **allDFs** have the same columns? You can not append if they don't. 

In [ ]:
# amount of rows and columns:
for df in allDFs:
    print(df.shape)

As we suspect this would get harder, let's make a list of column names:

In [ ]:
allColumnNames=[] # I will write every column 
for df in allDFs:
    allColumnNames.append(set(df.columns))# list of sets!

# this is what we have
allColumnNames

As we have an extra column in a couple of years, let's find first the common columns:

In [ ]:
# common columns
commonColumns=set.intersection(*allColumnNames) # expanding list of sets (*)
commonColumns

These are the columns not in the common names:

In [ ]:
commonColumns.symmetric_difference(set.union(*allColumnNames))

We decide here to leave that one out, so we could make a list of data frames with only the common columns:

In [ ]:
allDFs_sameNames=[] # final DataFrame (with all DFs from 2013-2021
colnamesSorted=sorted(list(commonColumns)) # columns names sorted - must turn 'set' into 'list'

# making list of DFs
for df in allDFs:
    allDFs_sameNames.append(df.loc[:,colnamesSorted]) 

Appending in pandas requires a list of data frames, in these case that is **allDFs_sameNames**. Then we proceed:

In [ ]:
# appending
allDFsConcat=pd.concat(allDFs_sameNames,ignore_index=True) # appending DFs using 'concat()'

#done!... see it:
allDFsConcat

We could pay attention to the current data types:

In [ ]:
allDFsConcat.info()

The columns **Year** and **Rank** were expected to be of numeric type, but we got the _object_ type instead. Let's explore *Year*:

In [ ]:
# value_counts can be used in object type
allDFsConcat.Year.value_counts()

Except for the year 2021, the other values are in date-time format. We just need an integer number, then:

In [ ]:
# keeping just the year value
yearAsNumber=[]
for y in allDFsConcat.Year:
    try:
        yearAsNumber.append(y.year)# the value from a date-time format
    except:
        yearAsNumber.append(y) # if not a datetime

#verifying
pd.Series(yearAsNumber).value_counts()

In [ ]:
# overwriting the year column
allDFsConcat['Year']=yearAsNumber

Have you noticed the column ordering?

In [ ]:
allDFsConcat.columns.to_list()

 Let's move 'Country','Year','Total' to the left:

In [ ]:
# this is a trick: setting columns as index
allDFsConcat.set_index(['Country','Year','Total'],inplace=True)
allDFsConcat.head()

Since I will not use _Rank_, I will get rid of it:

In [ ]:
# dropping unneeded column
allDFsConcat.drop(columns='Rank',inplace=True)

Now put the row indexes back:

In [ ]:
# indexes will be columns
allDFsConcat.reset_index(drop=False,inplace=True)

# see
allDFsConcat.head()

Let's do some cleaning on the column names:

In [ ]:
# see column names
allDFsConcat.columns.to_list()

In [ ]:
# clean column names
allDFsConcat.columns=allDFsConcat.columns.str.replace(':\s',"_",regex=True)
allDFsConcat.columns=allDFsConcat.columns.str.replace('\s',"",regex=True)
#see
allDFsConcat.columns.to_list()

Let's set the country names into upper case:

In [ ]:
# overwriting country
allDFsConcat['Country']=allDFsConcat.Country.str.upper()

Let's check again the format:

In [ ]:
allDFsConcat.info()

The **allDFsConcat** looks ok. Let's go into reshaping, where we will find more things to solve.

[Home](#home)

______

<a id='reshaping'></a>

## Reshaping

Data frames have have different shapes. Let me keep some columns from the last DF so you can notice something:

In [ ]:
# seeing long shape
fragileLong=allDFsConcat.iloc[:,:3]
fragileLong

You notice a DF is in long shape when you see the unit of analysis repeated in more than one cell.

In [ ]:
# country names repeated...
fragileLong.sort_values(['Country'])

Let me turn our **long** into **wide** shape:

In [ ]:
# to wide
fragileWide=pd.pivot_table(fragileLong,
               values='Total', # values to use
               index=['Country'], # unit of analysis
               columns=['Year']) # the values for NEW column
# see wide
fragileWide.head()

The **wide** shape from a **pivot_table** function looks great, but pay attention:

In [ ]:
fragileWide.index.names,fragileWide.columns.names,

As you see above, the indexes and columns have names. If you prefer not to keep those:

In [ ]:
# resetting index, keeping last index as a column
fragileWide= fragileWide.reset_index(drop=False).\
             rename_axis(index=None, columns=None) # no name for indexes

# result:
fragileWide.head()

Notoce that *Long shape* is efficient in the presence of missing values:

In [ ]:
# missing values in long format
fragileLong[fragileLong.isna().any(axis=1)]

But, *Wide shape* may create missing values after reshaping:

In [ ]:
# what cells have missing values?
fragileWide[fragileWide.isna().any(axis=1)]

This last request shows us problems.
As you can see, even though the DFs were prepared by the same organization, the DFs have country names that differ among different years. Here we need some **manual** changes: 

In [ ]:
# prepare changes as dict:
changes={"CABO VERDE": "CAPE VERDE",
"CZECHIA":"CZECH REPUBLIC",
"SWAZILAND":"ESWATINI",
"ISRAEL AND WEST BANK":"ISRAEL",
"KYRGYZSTAN":"KYRGYZ REPUBLIC",
"NORTH MACEDONIA":"MACEDONIA",
"SLOVAKIA": "SLOVAK REPUBLIC"}

Notice I am making the changes in **allDFsConcat** before re creating the long shape DF:

In [ ]:
# make changes using 'replace':
allDFsConcat.Country.replace(to_replace=changes,inplace=True)
# re create:
fragileLong=allDFsConcat.iloc[:,:3]

Let's redo the wide reshape:

In [ ]:
# to wide shape again
fragileWide=pd.pivot_table(fragileLong,
               values='Total',
               index=['Country'],
               columns=['Year']).\
            reset_index(drop=False).\
            rename_axis(index=None, columns=None)

# verify missing
fragileWide[fragileWide.isna().any(axis=1)] # remember you had an extra country

You have to be very careful when working with countries, specially when you are including or excluding countries; which may cause you hurting someone else's feelings. 

However, should we drop rows, we should reset the row indexes.

In [ ]:
fragileWide.reset_index(drop=True,inplace=True) 

For sure, we can turn this wide shape into a long one, using the function **melt**:

In [ ]:
pd.melt(fragileWide, id_vars=['Country'])

This would be more explicit:

In [ ]:
fragileLong=pd.melt(fragileWide, #DF
        id_vars=['Country'], #key
        value_vars=list(range(2013,2022)), # columns in wide
        var_name='Year', # new name for long column
        value_name='Total')# new name for values

# verifying
fragileLong.head()

As you just saw, while doing the reshaping we solved more problems in the **allDFsConcat**. Let's move into scaling.

[Home](#home)


______

<a id='scaling'></a>


## Data Scaling

It all look great so far. However, once you think you have the data ready, you should see the data ranges:

In [ ]:
allDFsConcat.describe(include='all')

The **describe** will only show numerical stats by default, so you need the parameter _include_ set to *all*. However, for our case, we should just request the range:

In [ ]:
allDFsConcat.iloc[:,2:].describe().loc[['min','max']].T # notice the transposing:

A boxplot may also be helpful:

In [ ]:
import matplotlib.pyplot as plt

allDFsConcat.iloc[:,2:].plot(kind='box', rot=90)
#plt.semilogy();

### Min-Max Scaling

As you see above, the range of Total is very different from the rest. Let's make sure that this column shares the same range as the others:

In [ ]:
columnsToScale=['Total'] # you can add more columns

from sklearn import preprocessing

# prepare the process
minmaxSc = preprocessing.MinMaxScaler(feature_range=(0, 10))# default is 0,1

# apply process
minmaxResult = minmaxSc.fit_transform(allDFsConcat[columnsToScale])

# result
minmaxResult

In [ ]:
# new values to new column
allDFsConcat['Total_minMax']=minmaxResult

In [ ]:
allDFsConcat.iloc[:,3:].plot(kind='box', rot=90)

Another way would be to standardize all these columns:

In [ ]:
allDFsConcat.iloc[:,3:-1]

### Standardization Scaling

In [ ]:
# prepare the process
stdSc = preprocessing.StandardScaler()

# apply process
stdScResult = stdSc.fit_transform(allDFsConcat.iloc[:,2:-1])

# result
stdScResult

In [ ]:
# numpy as pandas
pd.DataFrame(stdScResult)

In [ ]:
pd.DataFrame(stdScResult).plot(kind='box', rot=90)

we need names for those columns:

In [ ]:
newNames_sd=[name+'_sd' for name in allDFsConcat.iloc[:,2:-1].columns]
newNames_sd

Let me use that array to replace my values in the pandas _Series_:

In [ ]:
pd.DataFrame(stdScResult,columns=newNames_sd)

In [ ]:
# new data frame
stDF=pd.DataFrame(stdScResult,columns=newNames_sd)

# append to the end (right)
allDFsConcat=pd.concat([allDFsConcat,stDF],axis=1)

Now, these are my new data values:

In [ ]:
allDFsConcat.iloc[:,3:].plot(kind='box', rot=90);

The new columns have different values:

In [ ]:
allDFsConcat.loc[:,['Total','Total_minMax','Total_sd']].head()

But are prefectly correlated:

In [ ]:
# table
allDFsConcat.loc[:,['Total','Total_minMax','Total_sd']].corr()

In [ ]:
# plot
pd.plotting.scatter_matrix(allDFsConcat.loc[:,['Total','Total_minMax','Total_sd']], alpha=0.2)

Let me save **allDFsConcat**:

In [ ]:
allDFsConcat.to_csv(os.path.join("data","allDFsConcat.csv"))

[Home](#home)

______

<a id='merging'></a>

## Merging

Merging data sets need the following considerations:

* Merging is done on two data frames.
* You need columns in each data frame that share the same exact and unique values. The column names or titles need not be the same. In general, it is only one, but a combination of columns is possible.
* The merged table shows by default the mutual coincidences; but you can also request the values not matched, which will help you detect possible extra cleaning.
* Pandas jargon uses a **left** and a **right** data frame: **left**.merge(**right**).

Let me keep the data for 2021 in our fragility data frame:

In [ ]:
fragility2021=allDFsConcat[allDFsConcat.Year==2021]
fragility2021.drop(columns='Year',inplace=True)
fragility2021.reset_index(drop=True,inplace=True)
fragility2021.head()

Now, let me bring another data from the web with [country codes](https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes):

In [ ]:
# make sure to install 'html5lib', 'beautifulSoup4' and 'lxml'

codesLink='https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes'

allTablesWiki=pd.read_html(codesLink, flavor='bs4')

The object **allTablesWiki** is a list of data frames:

In [ ]:
# what and how many
type(allTablesWiki), len(allTablesWiki)

We have five DFs. Which is the one we need?

In [ ]:
# just guessing
allTablesWiki[0]

That was the right guess!

Now, let's keep that one:

In [ ]:
countryCodes=allTablesWiki[0].copy()

Let's check the column names:

In [ ]:
countryCodes.columns

Notice they are **MultiIndex**. Let's flatten the columns names:

In [ ]:
# option 1
['_'.join(col) for col in countryCodes.columns.values]

In [ ]:
# option 2
[col[1] for col in countryCodes.columns.values]

In [ ]:
#keeping option 2
countryCodes.columns=[col[1] for col in countryCodes.columns.values]
countryCodes.columns

The current names are not nice yet. Let's use some functions to make them look nicer: 

In [ ]:
# divide
countryCodes.columns.str.split("[")

In [ ]:
# divide and keep
[element[0] for element in countryCodes.columns.str.split("[")]

In [ ]:
# divide,  keep and titling
[element[0].title() for element in countryCodes.columns.str.split("[")]

In [ ]:
# divide,  keep , titling and replace
[element[0].title().replace(" ","") for element in countryCodes.columns.str.split("[")]

In [ ]:
# divide,  keep , titling and replace (and more replace)
[element[0].title().replace(" ","").replace("-","") for element in countryCodes.columns.str.split("[")]

The last version looks good enough. However, this might have been easier using **REGEX**:

In [ ]:
countryCodes.columns.str.title().str.replace('\[\w+\]|\s|\-',"",regex=True)

Let's make the change:

In [ ]:
countryCodes.columns=countryCodes.columns.str.title().str.replace('\[\w+\]|\s|\-',"",regex=True)

Let's drop some columns and rename the code columns:

In [ ]:
countryCodes.drop(columns=["SubdivisionCodeLinks"],inplace=True)
countryCodes.rename(columns={'Alpha2Code':'iso2','Alpha3Code':'iso3','InternetCctld':'internet'},inplace=True)
countryCodes.head()

Let me create a new column, but without accents and with country names in upper case:

In [ ]:
# bye accents
countryCodes['Country']=countryCodes['CountryName'].str.normalize('NFKD').\
                        str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper()

Let's see the current situation:

In [ ]:
countryCodes.info()

There is a row with missing values in iso2:

In [ ]:
countryCodes[countryCodes.isna().any(axis=1)]

Pandas interpreted the iso2 of a country as a missing value:

In [ ]:
countryCodes.loc[countryCodes.Country=='NAMIBIA','iso2']

In [ ]:
# easy fix
countryCodes.loc[countryCodes.Country=='NAMIBIA','iso2']="NA"

# something missing?
countryCodes[countryCodes.isna().any(axis=1)]

We could verify the amount of characters in iso2:

In [ ]:
# are these iso2 valid values?
[x for x in countryCodes.iso2 if len(x)>2]

That should not be possible. Let's check those rows:

In [ ]:
badValues=[x for x in countryCodes.iso2 if len(x)>2]

countryCodes[countryCodes.iso2.isin(badValues)]

Let's get rid of those rows:

In [ ]:
countryCodes=countryCodes[~countryCodes.iso2.isin(badValues)] # filtering

countryCodes.reset_index(drop=True,inplace=True) # needed when rows are dropped

We can continue with merging. But before that, keep these numbers in mind:

In [ ]:
fragility2021.shape,countryCodes.shape

Let me show you some merge approaches, but I will only show the amount of columns produced:

1. You keep only what is common in both key columns:

This is the default. The final rows will be the ones where the key values in each data frame match exactly. In this case, your count of rows will be at most the amount of rows of the smallest data frame.

In [ ]:
# how many resulting rows after inner merging
fragility2021.merge(countryCodes,how='inner',left_on='Country',right_on='Country').shape

2. You keep all the keys from one data frame:

The final rows will be all the rows from the dataframe (here from the _left_). If a key values does not find a match, the key value is kept, but the columns will have missing values. In this case, your count of rows will be equal to the amount of rows of the data frame to the left. You can also use **right** so the same logic applies to the data frame to the right.



In [ ]:
# how many resulting rows after left merging
fragility2021.merge(countryCodes,how='left',left_on='Country',right_on='Country').shape

3. You keep all the rows from both data frames:

In this case you will obtain all possible rows: the matched values, and the unmatched values from both data frames. You will also generate missing values. In this case, your count of rows will be at least the amount of rows of the data frame with the most rows.


In [ ]:
# how many resulting rows after outer merging
fragility2021.merge(countryCodes,how='outer',left_on='Country',right_on='Country').shape

Let's keep the **inner** merge on the CIA data files this time:

In [ ]:
# the default is inner merge
fragility2021_iso=fragility2021.merge(countryCodes) # notice less argument
fragility2021_iso

### Fuzzy Merge

Merging is done. 

Can we improve it?

We use fuzzy merging when we know that we could improve merge by changing the values in the key column so that more rows are matched. However, the algorithm can get confused if we have **noise** in the data. Let's look for noise:

Let's pay attention to the **Sovereignty** column:

In [ ]:
# maybe there are repeated countries because some territories are beyond their borders..?
countryCodes.Sovereignty.value_counts()

Do we have duplicates in column names?

In [ ]:
len(pd.unique(countryCodes.Country)),len(countryCodes.Country)

In [ ]:
countryCodes[countryCodes.Country.str.contains('UNITED|FRANCE|NETHER|AUSTRALIA|CHINA')]

Notice that we will look for **similarities**, so let's solve the USA case.

In [ ]:
countryCodes.loc[236,'Country']

In [ ]:
fragility2021[fragility2021.Country.str.contains('UNITED STATES')]

In [ ]:
#then
countryCodes.loc[236,'Country']='UNITED STATES'

Let's find what are the country names that are not shared between those DFs:

In [ ]:
# Countries in 'countryCodes' but NOT in 'fragility2021' 
OnlyCodes=set(countryCodes.Country)-set(fragility2021.Country)
OnlyCodes

In [ ]:
# Countries in 'fragility2021'  but NOT in 'countryCodes'
OnlyFragility=set(fragility2021.Country)-set(countryCodes.Country)
OnlyFragility

Here, we should try to find what countries in _OnlyFragility_ may match the ones in _OnlyCodes_. 

We need to use the **fuzzy merge** approach (please install **thefuzz** if not previously installed):

In [ ]:
from thefuzz import process as fz

# take a country from OnlyFragility
# look for a country in OnlyCodes and return the most similar
[(f,fz.extractOne(f, OnlyCodes)) for f in sorted(OnlyFragility)]

Above you have found _some_ good matches. Let's keep the best ones:

In [ ]:
[(f,fz.extractOne(f, OnlyCodes)) for f in sorted(OnlyFragility)
 if fz.extractOne(f, OnlyCodes)[1]>=90]

Once you have good matches, you have to create a dictionary like this:

In [ ]:
changesCodes1={fz.extractOne(f, OnlyCodes)[0]:f 
                 for f in sorted(OnlyFragility)
                 if fz.extractOne(f, OnlyCodes)[1] >=90}
#dict of matches
changesCodes1

You can use that dict for the replacements in *countryCodes*:

In [ ]:
countryCodes.Country.replace(to_replace=changesCodes1,inplace=True)


This process can be done a few more times, and you can recover more rows for the merging process. Let's see:

In [ ]:
# second try
OnlyCodes=set(countryCodes.Country)-set(fragility2021.Country)
OnlyFragility=set(fragility2021.Country)-set(countryCodes.Country)
[(f,fz.extractOne(f, OnlyCodes)) for f in sorted(OnlyFragility)]

We might need some manual changes:

In [ ]:
countryCodes[countryCodes.Country.str.contains('LAO|KOREA|CZECH',regex=True)]

The presence of repetitive strings confuses the fuzzy algorithm. Then, we could get rid of those:

In [ ]:
# solving manually KOREA:
countryCodes.loc[118,'Country']='NORTH KOREA'
countryCodes.loc[119,'Country']='SOUTH KOREA'
countryCodes.loc[122,'Country']='LAOS'
countryCodes.loc[59,'Country']='CZECH REPUBLIC'

In [ ]:
# again
OnlyCodes=set(countryCodes.Country)-set(fragility2021.Country)
OnlyFragility=set(fragility2021.Country)-set(countryCodes.Country)
[(f,fz.extractOne(f, OnlyCodes)) for f in sorted(OnlyFragility)]

In [ ]:
# second dict of changes
changesCodes2={fz.extractOne(f, OnlyCodes)[0]:f 
                 for f in sorted(OnlyFragility)
                 if fz.extractOne(f, OnlyCodes)[1] >=52}
#dict of matches
changesCodes2

In [ ]:
# make the changes
countryCodes.Country.replace(to_replace=changesCodes2,inplace=True)

In [ ]:
# last try?
OnlyCodes=set(countryCodes.Country)-set(fragility2021.Country)
OnlyFragility=set(fragility2021.Country)-set(countryCodes.Country)
[(f,fz.extractOne(f, OnlyCodes)) for f in sorted(OnlyFragility)]

We are ready for the merge:

In [ ]:
fragility2021_iso=fragility2021.merge(countryCodes) #merge on Country
fragility2021_iso

We have recovered all the rows to match the amount of rows in *fragility2021*!

In [ ]:
#checking:
fragility2021_iso.info()

### Geo Merging

Let me fetch a world map from this 
[website](https://public.opendatasoft.com/explore/dataset/world-administrative-boundaries/export/). Download the **geojson** format and save it your *maps* folder.

In [ ]:
import geopandas as gpd
import os

world=gpd.read_file(os.path.join("maps","world-administrative-boundaries.geojson"))

In [ ]:
world.info()

In [ ]:
# our file
world.plot()

In [ ]:
# the crs info
world.crs

In [ ]:
# any missing?
world[world.isna().any(axis=1)]

In [ ]:
# map with no missing:

worldFull=world[~world.isna().any(axis=1)]

When you merge a GDF with a DF, **the GDF has to be on the left**:

In [ ]:
theMap=worldFull.merge(fragility2021_iso,on='iso3')
# here it is:
theMap

Notice your merge brought two more rows!

In [ ]:
# duplicates?
theMap[pd.Series(theMap.iso3).duplicated()]

In [ ]:
# more detail
theMap[theMap.iso3.str.contains('PSE|PRT')]

In [ ]:
# take a look:
theMap[theMap.iso3.str.contains('PSE|PRT')].explore()

Let's save what we have:

In [ ]:
theMap.to_file(os.path.join("maps","worlMapData.gpkg"), layer='countries', driver="GPKG")

It is time to color our maps!

## Choropleths

Let me take a look at the Total variable (_Total_minMax_):

In [ ]:
theMap['Total_minMax'].describe()

In [ ]:
theMap.Total_minMax.hist()

This plot tells you the distribution of the values, but not the presence of outliers, which you are revealed in a boxplot:

In [ ]:
theMap.boxplot(column=['Total_minMax'])  

Notice the histogram divides the data in intervals which are the base of the bars. Seaborn uses the [Freedman-Diaconis](https://en.wikipedia.org/wiki/Freedman%E2%80%93Diaconis_rule) formula to compute the bins.

Let's see other possibilities, but please install [**numba**](https://numba.readthedocs.io/en/stable/user/installing.html) before runing the next code; also make sure you have **pysal**, **mapclassify** and **numpy** installed: 

In [ ]:
import mapclassify 
import numpy as np

np.random.seed(12345) # so we all get the same results!

# let's try 5 intervals
K=5
theVar=theMap.Total_minMax
# same interval width, easy interpretation
ei5 = mapclassify.EqualInterval(theVar, k=K)
# same interval width based on standard deviation, easy - but not as the previous one, poor when high skewness
msd = mapclassify.StdMean(theVar)
# interval width varies, counts per interval are close, not easy to grasp, repeated values complicate cuts                                
q5=mapclassify.Quantiles(theVar,k=K)

# based on similarity, good for multimodal data 
mb5 = mapclassify.MaximumBreaks(theVar, k=K)
# based on similarity, good for skewed data
ht = mapclassify.HeadTailBreaks(theVar) # no K needed
# based on similarity, optimizer
fj5 = mapclassify.FisherJenks(theVar, k=K)
# based on similarity, optimizer
jc5 = mapclassify.JenksCaspall(theVar, k=K)
# based on similarity, optimizer
mp5 = mapclassify.MaxP(theVar, k=K) 

Let's see the **HeadTailBreaks** results:

In [ ]:
# a frequency table by default
ht

In [ ]:
# group label
ht.yb

In [ ]:
# labels and counts
np.unique(ht.yb,return_counts=True)

In [ ]:
# ht.yb into a pandas Series

pd.Series(ht.yb).value_counts()

In [ ]:
# these are the cuts, but the min value is not including 
ht.bins

In [ ]:
# completing the bins
HT_bins=list(ht.bins)
HT_bins.insert(0,theVar.min())

In [ ]:
theMap.Total_minMax.hist(bins=HT_bins)

How can we select the right classification?
Let me use the the Absolute deviation around class median (ADCM) to make the comparisson:

In [ ]:
class5 = q5, ei5,msd, ht, mb5, fj5, jc5, mp5
# Collect ADCM for each classifier
fits = np.array([ c.adcm for c in class5])
# Convert ADCM scores to a DataFrame
adcms = pd.DataFrame(fits)
# Add classifier names
adcms['classifier'] = [c.name for c in class5]
# Add column names to the ADCM
adcms.columns = ['ADCM', 'Classifier']

Now, plot the **adcms**:

In [ ]:
adcms.sort_values('ADCM').plot.barh(x='Classifier')

Let me keep the three best options:

In [ ]:
theMap['Total_ei5'] = ei5.yb
theMap['Total_fj5'] = fj5.yb
theMap['Total_jc5'] = jc5.yb

# we have:
theMap[['Total_ei5','Total_fj5','Total_jc5']].info()

This is how many countries we have per class, in each scheme:

In [ ]:
class5new = ei5,fj5, jc5
pd.DataFrame(
    {c.name: c.counts for c in class5new},
    index=['Class-{}'.format(i) for i in range(5)]
)

Let's plot _Total_ei5_:

In [ ]:
import matplotlib.pyplot as plt

f, ax = plt.subplots(1, figsize=(10, 10))
theMap.plot(column='Total_ei5', 
        cmap='viridis', 
        categorical=True,
        edgecolor='white', 
        linewidth=0., 
        alpha=0.75, 
        legend=True,
        legend_kwds={'loc':3},
        ax=ax
       )

ax.set_axis_off()

Let's plot all the schemes:

In [ ]:
# learn "enumerate()":
varsToPlot = ['Total_ei5','Total_fj5','Total_jc5']
[x for x in enumerate(varsToPlot)]

In [ ]:
# create 3 figures
f, axs = plt.subplots(nrows=3, ncols=1, figsize=(12, 12))

axs = axs.flatten() # just a one dimensional index

for i, col in enumerate(varsToPlot):    
    HERE = axs[i]# select the axis where the map will go
    theMap.plot(column=col, categorical=True, linewidth=0.1,
                legend=True,cmap='hot_r',edgecolor='grey',
                legend_kwds={'loc':3},
                ax=HERE)
    # Remove axis clutter
    ax.set_axis_off()
    # Set the axis title to the name of variable being plotted
    ax.set_title(col)
# Display the figure
plt.show()

In [ ]:
# saving
theMap.to_file(os.path.join("maps","worlMapData.gpkg"), layer='countries', driver="GPKG")

Let's keep one continent:

In [ ]:
theMap.continent.value_counts()

In [ ]:
americaMap=theMap[theMap.continent=='Americas']

f, ax = plt.subplots(1, figsize=(12, 8))
americaMap.plot(ax=ax)

ax.set_axis_off()

In [ ]:
# is projected

americaMap.crs.axis_info

In [ ]:
# reprojecting
americaMap=americaMap.to_crs(8858)

Let's request the centroid of every country:

In [ ]:
americaMap['centroid']=americaMap.geometry.centroid
americaMap.head()

In [ ]:
americaMap.info()

In [ ]:
# just the plot
base=americaMap.plot(facecolor='white',
                     edgecolor='lightgrey',
                     linewidth=0.3,
                     figsize=(8,8))

# another column, with the size of the POINTS (based on "Total_ei5")
americaMap["sizeOfMarker_ei5"]=[4**exp for exp in americaMap["Total_ei5"]]

# now, plotting the centroids (declared as the geometry)!!!
americaMap.set_geometry("centroid").plot(column="Total_ei5",
                                         markersize=americaMap["sizeOfMarker_ei5"],
                                         cmap='Paired_r',
                                         legend=True,ax=base)

In [ ]:
# facetting
f, axs = plt.subplots(nrows=2, ncols=3, figsize=(8, 8))
axs = axs.flatten()

for i in range(5):  
    ax = axs[i]# select the axis where the map will go
    americaMap.plot(ax=ax,color='gainsboro')
    americaMap[americaMap.Total_ei5==i].plot(ax=ax,color='red')
    ax.set_axis_off()
    ax.set_title("level_"+str(i))
f.delaxes(axs[5]) # bye empty subplot
plt.show()